# LangChain Tree of Thoughts (ToT)
## Manual approach (via prompts)

### Init

In [13]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(),verbose=True) # read local .env file

#  for the langchain
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_TYPE= os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_ID = os.getenv("OPENAI_DEPLOYMENT_ID")

In [14]:
from langchain.chains import LLMChain
# from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [15]:
# to disable SSL verification that causes problem from my laptop
from ssl_workaround import no_ssl_verification

### Prompts

In [16]:
template ="""
Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template                      
)

chain1 = LLMChain(
    llm=ChatOpenAI(
        temperature=0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        engine = OPENAI_DEPLOYMENT_ID),
    prompt=prompt,
    output_key="solutions"
)

template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template                      
)

chain2 = LLMChain(
    llm=ChatOpenAI(
        temperature=0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        engine = OPENAI_DEPLOYMENT_ID),
    prompt=prompt,
    output_key="review"
)

template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template                      
)

chain3 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        engine = OPENAI_DEPLOYMENT_ID),
    prompt=prompt,
    output_key="deepen_thought_process"
)

template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template                      
)

chain4 = LLMChain(
    llm=ChatOpenAI(
        temperature=0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        engine = OPENAI_DEPLOYMENT_ID),
    prompt=prompt,
    output_key="ranked_solutions"
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [18]:
chain1
with no_ssl_verification():
    response = chain1.predict(input = "human colonization of Mars",
                            perfect_factors = "The distance between Earth and Mars is very large, making regular resupply difficult")
print(response)

1. Develop sustainable and self-sufficient habitats on Mars: One solution could be to focus on developing habitats that are capable of sustaining human life on Mars without the need for regular resupply from Earth. This could involve using local resources such as water and minerals to create food, oxygen, and other necessities for survival.

2. Establish a regular supply chain: Another solution could be to establish a regular supply chain between Earth and Mars, using advanced spacecraft and logistics systems to transport essential supplies and equipment to the planet. This could involve developing new technologies such as reusable rockets and advanced robotics to make the process more efficient and cost-effective.

3. Develop a hybrid approach: A third solution could be to combine the above two approaches, by developing sustainable habitats on Mars while also establishing a regular supply chain from Earth. This could involve using the supply chain to transport critical supplies and eq

In [21]:
from langchain.chains import SequentialChain
import langchain
langchain.debug = True

In [20]:
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

with no_ssl_verification():
    solution = overall_chain({"input":"human colonization of Mars", "perfect_factors":"The distance between Earth and Mars is very large, making regular resupply difficult"})



> Entering new SequentialChain chain...

> Finished chain.


In [22]:
print(solution)

{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Based on the evaluations and scenarios, the solutions can be ranked in the following order of promise:\n\n1. Developing a hybrid approach\n2. Establishing a regular supply chain\n3. Developing sustainable and self-sufficient habitats on Mars\n\nThe hybrid approach is ranked as the most promising solution because it combines the benefits of both sustainable habitats and a regular supply chain. This approach allows for long-term habitation on Mars while also ensuring a steady supply of essential resources and equipment. The potential obstacles of coordinating and integrating the two approaches can be overcome with partnerships and research and development investments. The unexpected outcomes of this approach could lead to new technologies and methods for integrating sustainable habitats and space transportation that could 

### Sequential chain - another one for movies

In [5]:
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate

In [6]:
llm = ChatOpenAI(
        temperature=0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        engine = OPENAI_DEPLOYMENT_ID)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [7]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [8]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [9]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [10]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [11]:
with no_ssl_verification():
    outcome = overall_chain("tohle je kratke review. film je super.")



> Entering new SequentialChain chain...

> Finished chain.


In [12]:
outcome

{'Review': 'tohle je kratke review. film je super.',
 'English_Review': 'This is a short review. The movie is great.',
 'summary': 'The movie is great.',
 'followup_message': 'Shrnutí: Film je skvělý.\n\nReakce: To je skvělé! Mám rád, když se lidem líbí filmy. Můžeš mi říct více o tom, co se ti na filmu líbilo? Byla to dobrá zábava nebo měl film hlubší poselství? Rád bych slyšel více podrobností o tom, co dělá tento film tak skvělým.'}